In [2]:
import numpy as np
import torch
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm
import torchvision
from torchvision.transforms import v2
from torch.utils.data import DataLoader
from torch import nn

ModuleNotFoundError: No module named 'tqdm'

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.manual_seed(0xC0FFEE)


train_transform = v2.Compose([
    v2.RandomHorizontalFlip(p=0.5),
    v2.RandomCrop(size=32, padding=4),
    v2.ToTensor(),
    v2.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])



test_transform = v2.Compose([
    v2.ToTensor(),
    v2.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)) 
])

batch_size = 40

train_set = torchvision.datasets.CIFAR10(root = './data/', train = True, transform = train_transform, download = True)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)


test_set = torchvision.datasets.CIFAR10(root = './data/', train = False, transform = test_transform, download = True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)


classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

print('Train size', len(train_set))
print('Test size', len(test_set))

c:\Users\Administrator\Desktop\skoltech_work\hbf_clustering\.venv\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.
  warnings.warn(


Files already downloaded and verified
Files already downloaded and verified
Train size 50000
Test size 10000


In [3]:
def epoch_train(loader, clf, criterion, opt):
    # BEGIN Solution (do not delete this comment!)
    clf.train(True)
    
    loss = 0
    correct = 0
    N = 0
    
    for train_images, train_labels in loader:
        train_images = train_images.to(device)
        train_labels = train_labels.to(device)
        
        opt.zero_grad()
        y_preds = clf(train_images)
        train_loss = criterion(y_preds, train_labels)
        train_loss.backward()
        opt.step()
        
        correct += (torch.argmax(y_preds, dim=1) == train_labels).sum().item()
        loss += train_loss.item()
        N += train_labels.size(0)
    # END Solution (do not delete this comment!)
    return loss/len(loader), correct / N

def epoch_test(loader, clf, criterion):
    # BEGIN Solution (do not delete this comment!)
    clf.eval() 

    loss = 0
    correct = 0
    N = 0

    for test_images, test_labels in loader:
        test_images = test_images.to(device)
        test_labels = test_labels.to(device)
    
        y_preds = clf(test_images)
        test_loss = criterion(y_preds, test_labels)

        correct += (torch.argmax(y_preds, dim=1) == test_labels).sum().item()
        loss += test_loss.item()
        N += test_labels.size(0)

    # END Solution (do not delete this comment!)
    return loss/len(loader), correct/N

# The function which you are going to use for model training
def train(train_loader, test_loader, clf, criterion, opt, n_epochs=50):
    for epoch in (range(n_epochs)):
        train_loss, train_acc = epoch_train(train_loader, clf, criterion, opt)
        test_loss, test_acc = epoch_test(test_loader, clf, criterion)

        print(f'[Epoch {epoch + 1}] train loss: {train_loss:.3f}; train acc: {train_acc:.2f}; ' + 
              f'test loss: {test_loss:.3f}; test acc: {test_acc:.2f}')
        
def test(test_loader, clf, criterion):
        test_loss, test_acc = epoch_test(test_loader, clf, criterion)
        print(f'test loss: {test_loss:.3f}; test acc: {test_acc:.2f}')

In [4]:
class MLP(nn.Module):
    def __init__(self, num_classes=10):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32 * 32 * 3, 1000),
            nn.BatchNorm1d(1000),
            nn.ReLU(),
            nn.Linear(1000, 500),
            nn.BatchNorm1d(500),
            nn.ReLU(),
            nn.Linear(500, 300),
            nn.BatchNorm1d(300),
            nn.ReLU(),
            nn.Linear(300, 100),
            nn.BatchNorm1d(100),
            nn.ReLU(),
            nn.Linear(100, num_classes)
        )
        
    def forward(self, x):
        return self.layers(x)
    


In [5]:
clf_mlp = MLP(num_classes=10).cuda()
print('Number of weights:', np.sum([np.prod(p.shape) for p in clf_mlp.parameters()]))

# Check that the output size of the network is BATCH_SIZE x NUM_CLASSES
X = next(iter(train_loader))[0].cuda()
with torch.no_grad():
    clf_X = clf_mlp(X)
    assert len(clf_X) == len(X)
    assert clf_X.shape[1] == 10

Number of weights: 3758710


In [6]:
opt = torch.optim.Adamax(clf_mlp.parameters(), lr=2e-3)
criterion = nn.CrossEntropyLoss()

train(train_loader, test_loader, clf_mlp, criterion, opt, n_epochs=5)

[Epoch 1] train loss: 1.828; train acc: 0.34; test loss: 1.579; test acc: 0.43
[Epoch 2] train loss: 1.662; train acc: 0.40; test loss: 1.533; test acc: 0.44
[Epoch 3] train loss: 1.582; train acc: 0.43; test loss: 1.441; test acc: 0.48
[Epoch 4] train loss: 1.527; train acc: 0.45; test loss: 1.407; test acc: 0.49
[Epoch 5] train loss: 1.488; train acc: 0.46; test loss: 1.375; test acc: 0.51


In [47]:
clf_mlp.layers[1].weight.type

<function Parameter.type>

In [53]:
def get_weights(clf):    

    weights = {}
    biases ={}
    for l_n, l_i in zip(['L_input', 'BN1', 'HL2', 'BN2', 'HL3', 'BN3','HL4', 'BN4', 'L_out'], [1,2,4,5,7,8,10,11,13]):
        weights[l_n] = clf.layers[l_i].weight.detach().cpu().numpy()
        biases[l_n] = clf.layers[l_i].bias.detach().cpu().numpy()
    return weights, biases


def set_weights(clf, weights, biases):    

    for l_n, l_i in zip(['L_input', 'BN1', 'HL2', 'BN2', 'HL3', 'BN3','HL4', 'BN4', 'L_out'], [1,2,4,5,7,8,10,11,13]):
        clf.layers[l_i].weight = torch.nn.parameter.Parameter(data=torch.from_numpy(weights[l_n]), requires_grad=True)
        clf.layers[l_i].bias = torch.nn.parameter.Parameter(data=torch.from_numpy(biases[l_n]), requires_grad=True) 
    return None


In [28]:
layers_names = ['L_input', 'BN1', 'HL2', 'BN2', 'HL3', 'BN3','HL4', 'BN4', 'L_out']
layers_idxs = [1,2,4,5,7,8,10,11,13]

In [49]:
a = get_weights(clf_mlp)
a[0]['L_input'][0][0] = 0
a

({'L_input': array([[ 0.        , -0.05443656, -0.07518243, ..., -0.05408807,
          -0.06809054, -0.07610493],
         [ 0.05444981, -0.00671054,  0.03119436, ..., -0.03283515,
          -0.02557418, -0.03926486],
         [-0.02053263,  0.01604585,  0.01990883, ..., -0.07309712,
          -0.06152126, -0.06686657],
         ...,
         [ 0.01537927,  0.01731919, -0.00214701, ..., -0.02114874,
           0.00130992, -0.06563611],
         [-0.00144658,  0.00129515,  0.04238188, ...,  0.12203601,
           0.12329025,  0.14908154],
         [ 0.08875178,  0.03488357,  0.04254657, ..., -0.01866012,
          -0.00036644, -0.00204669]], dtype=float32),
  'BN1': array([0.95298356, 0.97054535, 0.8766669 , 0.9238833 , 0.82331365,
         0.9150977 , 0.8821748 , 0.8652906 , 1.1185251 , 0.9411856 ,
         0.79025644, 1.0228444 , 0.9913546 , 1.0166332 , 0.83616406,
         1.1700562 , 0.9440232 , 1.1630077 , 0.74222887, 0.8784285 ,
         1.102768  , 0.9233284 , 0.8957939 , 0.8733

In [54]:
set_weights(clf_mlp, a[0], a[1])